Spark provides GraphX for graphs and graph-parallel computation. To support graph computation, GraphX extends Spark RDD and presents a set of fundamental operators (e.g., subgraph, joinVertices, and aggregateMessages) as well as an optimized variant of the Pregel API. This tutorial is going to get you started with the basics tasks, such as importing graphs in GraphX data strucutres as well as run some analytic tasks like connected components and triangle count.

In [1]:
import org.apache.spark._
import org.apache.spark.graphx._
import org.apache.spark.rdd.RDD
import org.apache.spark.graphx.GraphLoader
import org.apache.spark.sql.SparkSession

1. Some basics:
The property graph is a directed multigraph with user defined objects attached to each vertex and edge. Each vertex has a unique 64-bit long identifier (VertexId), while edges are identified by the corresponding source and destination vertex identifiers. The property graph is parameterized over the vertex (VD) and edge (ED) types. 

In [3]:
val vertexArray = Array(
  (1L, ("Alice", 28)),
  (2L, ("Bob", 27)),
  (3L, ("Jussi", 35)),
  (4L, ("Magnus", 42)),
  (5L, ("Michael", 53)),
  (6L, ("Martin", 40))
  )
val edgeArray = Array(
  Edge(2L, 1L, 7),
  Edge(2L, 4L, 2),
  Edge(3L, 2L, 4),
  Edge(3L, 6L, 3),
  Edge(4L, 1L, 1),
  Edge(5L, 2L, 2),
  Edge(5L, 3L, 8),
  Edge(5L, 6L, 3)
  )

val vertexRDD: RDD[(Long, (String, Int))] = sc.parallelize(vertexArray)
val edgeRDD: RDD[Edge[Int]] = sc.parallelize(edgeArray)
val graph: Graph[(String, Int), Int] = Graph(vertexRDD, edgeRDD)

graph.triplets.map(
  triplet => triplet.srcAttr._1 + " retweeted " + triplet.attr + " posts of " + triplet.dstAttr._1
  ).collect.foreach(println(_))

Bob retweeted 7 posts of Alice
Bob retweeted 2 posts of Magnus
Jussi retweeted 4 posts of Bob
Jussi retweeted 3 posts of Martin
Magnus retweeted 1 posts of Alice
Michael retweeted 2 posts of Bob
Michael retweeted 8 posts of Jussi
Michael retweeted 3 posts of Martin


In many cases we will want to extract the vertex and edge RDD views of a graph. As a consequence, the graph class contains members (graph.vertices and graph.edges) to access the vertices and edges of the graph. While these members extend RDD[(VertexId, V)] and RDD[Edge[E]] they are actually backed by optimized representations that leverage the internal GraphX representation of graph data. Below, graph.vertices is used to display the names of the users who are at least 30 years old.

In [4]:
graph.vertices.filter { case (id, (name, age)) => age >30 }.collect.foreach(v => println(s"${v._2._1} is ${v._2._2}"))

Magnus is 42
Michael is 53
Martin is 40
Jussi is 35


What would be the output of the following?


In [5]:
graph.edges.filter { case  Edge(src, dst, w) => w > 5 }.collect.foreach(e => println(s"from ${e.srcId} to ${e.dstId}"))

from 2 to 1
from 5 to 3


In [6]:
val inDegrees: VertexRDD[Int] = graph.inDegrees
inDegrees.collect.foreach(a => println(a))

(4,1)
(1,2)
(6,2)
(2,2)
(3,1)



2. Subgraph Operator
Suppose you want to study the community structure of subset of the nodes in the graph. To support this type of analysis GraphX includes the subgraph operator that takes vertex and edge predicates and returns the graph containing only the vertices that satisfy the vertex predicate (evaluate to true) and edges that satisfy the edge predicate and connect vertices that satisfy the vertex predicate.

In [7]:
val olderGraph = graph.subgraph(vpred = (id, attr) => attr._2 >30)

// compute the connected components
val cc = olderGraph.connectedComponents

// display the component id of each user:
olderGraph.vertices.leftJoin(cc.vertices) {
  case (id, user, comp) => s"${user._1} is in component ${comp.get}"
}.collect.foreach{ case (id, str) => println(str) }

Magnus is in component 4
Michael is in component 3
Martin is in component 3
Jussi is in component 3



3. Join Operators
In many cases it is necessary to join data from external collections (RDDs) with graphs. Let's incorporate the in and out degree of each vertex into the vertex property. To do this, we first define a User class to better organize the vertex property and build a new graph with the user property. We initialized each vertex with in/out degree =0. Then, we join the in and out degree information with each vertex building the new vertex property. Here we use the outerJoinVertices method of Graph that takes two argument lists: (i) an RDD of vertex values, and (ii) a function from the id, attribute, and Optional matching value in the RDD to a new vertex value. The outerJoinVertices has the following type signature:


In [21]:
// Define a class to more clearly model the user property
case class User(name: String, age: Int, inDeg: Int, outDeg: Int) 

// Create a user Graph
val initialUserGraph: Graph[User, Int] = graph.mapVertices{ case (id, (name, age)) => User(name, age, 0, 0) }

// Fill in the degree information
val outDegrees: VertexRDD[Int] = graph.outDegrees
val inDegrees: VertexRDD[Int] = graph.inDegrees

val userGraph = initialUserGraph.outerJoinVertices(initialUserGraph.inDegrees) {
  case (id, u, inDegOpt) => User(u.name, u.age, inDegOpt.getOrElse(0), u.outDeg)
}.outerJoinVertices(initialUserGraph.outDegrees) {
  case (id, u, outDegOpt) => User(u.name, u.age, u.inDeg, outDegOpt.getOrElse(0))
}

In [41]:
userGraph.vertices.filter { case (id, u) => u.inDeg >0 }.collect.foreach( u => println(s"${u._2} "))

User(Magnus,42,1,1) 
User(Alice,28,2,0) 
User(Martin,40,2,0) 
User(Bob,27,2,2) 
User(Jussi,35,1,2) 
